In [51]:
import lyricsgenius as lg
import pandas as pd
import numpy as np

In [52]:
df = pd.DataFrame(columns = ['Artist', 'Song', 'Lyrics','Genre'])
print(df)

Empty DataFrame
Columns: [Artist, Song, Lyrics, Genre]
Index: []


In [47]:
# file = open("lyrics.txt", "w")

genius = lg.Genius('Kazu51bV8gbOwO6RQpZA-ZQ-4iOWbGdiQADlWk6SRO6DGtoezXzRL6Qapzdpu-VJ', skip_non_songs=True, 
excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True)

pop = ['One Direction', 'Ariana Grande', 'Justin Bieber', 'Miley Cyrus', 'Justin Timberlake', 'Katy Perry', 'Black Eyed Peas', 'Demi Lovato', 'Meghan Trainor', 'Kesha', 
       'Lady Gaga', 'Shawn Mendes', 'Sam Smith', 'Camilla Cabello', 'Chainsmokers', 'Jason Derulo', 'Calvin Harris', 'Pink', '5 Seconds of Summer', 'The Weeknd', 
       'Harry Styles', 'Jennifer Lopez', 'Charlie Puth', 'Michael Jackson', 'Prince']
rap = ['Fetty Wap', 'Migos', '21 Savage', '88GLAM', 'Roddy Rich', 'Lil Baby', 'Eminem', 'Snoop Dogg', 'Drake','Kanye West', 'Travis Scott', 'Dr. Dre','Lil Wayne' ,
       '50 Cent' ,'Jay-Z', 'Kendrick Lamar' ,'J. Cole' ,'Nas' ,'Kid Cudi' ,'Cardi B', 'Nicki Minaj', 'A$AP Rocky' ,'Pop Smoke', 'Meek Mill', 'Young Thug']
conutry = ['Tim McGraw', 'Blake Shelton', 'George Strait', 'Garth Brooks', 'Luke Bryan', 'Luke Combs', 'Kenny Chesney', 'Keith Urban', 'Johnny Cash', 'Carrie Underwood', 
           'Brad Paisley', 'Merie Haggard', 'Jason Aldean', 'Dierks Bentley', 'Alan Jackson', 'Chris Stapleton', 'Kenny Rogers', 'Toby Keith', 'Darius Rucker', 'George Jones', 
           'Willie Nelson', 'Miranda Lambert', 'Chris Young', 'Randy Travis', 'Thomas Rhett']


def get_lyrics(df, artists, k):  # Write lyrics of k songs by each artist in arr
    c = 0  # Counter
    err = 0
    for name in artists.keys():
        while True:
            try:
                songs = (genius.search_artist(name, max_songs=k, sort='popularity')).songs
                for song in songs:
                    df = df.append({'Artist':song.artist , 'Song':song.title , 'Lyrics':song.lyrics , 'Genre':artists[name]}, ignore_index = True)
                break
            except:
                err += 1
                print('err',err)
                pass
            # except:  #  Broad catch which will give us the name of artist and song that threw the exception
            # print(f"some exception at {name}: {c}")
    
    return df

new_df = get_lyrics(df, artists, 2)

Searching for songs by Drake...

Song 1: "God’s Plan"
err 1
Searching for songs by Drake...

Song 1: "God’s Plan"
err 2
Searching for songs by Drake...

Song 1: "God’s Plan"
Song 2: "In My Feelings"

Reached user-specified song limit (2).
Done. Found 2 songs.


In [77]:
len(pop)

25